In [ ]:
!pip install geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 7.7 MB/s 


In [ ]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import geocoder

locations = ['Adyar','Agaram','Anna Nagar','Arumbakkam','Ashok Nagar','Guindy','Kodambakkam','Myapore','Royapet','Teynampet','Thygaraya Nagar','Taramani','Vadapalani','Velachery','Virugambakkam','Washermanpet'] 
kl_df = pd.DataFrame({"Neighborhood": locations})

address = 'Chennai, India'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, India {}, {}.'.format(latitude, longitude))

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(location) for location in kl_df["Neighborhood"].tolist()]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
print(kl_df.shape)
kl_df

The geograpical coordinate of Chennai, India 13.0836939, 80.270186.
(16, 3)


,Neighborhood,Latitude,Longitude
0,Adyar,13.00303,80.25186
1,Agaram,13.10956,80.23235
2,Anna Nagar,13.08360,80.21013
3,Arumbakkam,13.07308,80.20952
4,Ashok Nagar,13.03550,80.21218
5,Guindy,13.00415,80.22014
6,Kodambakkam,13.05152,80.22436
7,Myapore,13.03155,80.26022
8,Royapet,13.08375,80.28256
9,Teynampet,13.04819,80.24967


In [ ]:
import folium
#map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)
loc=str(input("Enter location: "))
for i in range(len(locations)):
  if(loc==locations[i]):
    coords=get_latlng(locations[i])
    label='{}'.format(locations[i])
    lat=coords[0]
    lng=coords[1]
map_kl = folium.Map(location=[lat, lng], zoom_start=11)
label = folium.Popup(label, parse_html=True)
folium.CircleMarker([lat, lng],radius=10,popup=label,color='red',fill=True,fill_color='#3186cc',fill_opacity=1).add_to(map_kl)
map_kl

Enter location: Washermanpet


In [ ]:
import pandas as pd # for data processing
import folium   #for creating maps
import requests  #for retreiving Information from URL
from geopy.geocoders import Nominatim  #converting address to cordinates
from pandas.io.json import json_normalize
import json

def list2string(s):
  str1=""
  for e in s:
    str1+=e
  return str1


radius = 2000
LIMIT = 50

URL= "https://api.foursquare.com/v3/places/search?ll={},{}&radius={}&limit={}"
headers = {
      "Accept": "application/json",
      "Authorization": "key"
    }

df_list=[]
    
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
      url = URL.format(lat, lng, radius, LIMIT)
      results = requests.request("GET", url, headers=headers).json()
        
      for each_result in results['results']: # filter the result based on JSON identification
            result={}
            result['Neighborhood']=neighborhood
            result['Neighborhood Latitude']=lat
            result['Neighborhood Longitude']=lng
            result['Name']=each_result['name']
            result['Venue Latitude']=each_result['geocodes']['main']['latitude']
            result['Venue Longitude']=each_result['geocodes']['main']['longitude']
            result['Category_Names']=list2string([each_name['name'] for each_name in each_result['categories']])
            df_list.append(result.copy())

venues_df=pd.DataFrame(df_list) # return dataframe
venues_df.columns=['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude','VenueCategory']
print(venues_df.shape)
venues_df.head()

(800, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adyar,13.00303,80.25186,"Nalli, Adayar",13.002887,80.252095,Women's Store
1,Adyar,13.00303,80.25186,Domino's Pizza,13.002041,80.254157,PizzeriaFast Food Restaurant
2,Adyar,13.00303,80.25186,Starbucks,13.001466,80.251897,Coffee Shop
3,Adyar,13.00303,80.25186,Wonton Restaurant,13.005190,80.251826,Restaurant
4,Adyar,13.00303,80.25186,That Madras Place,13.006001,80.250761,Restaurant


In [ ]:
venues_df.groupby(["Neighborhood"]).count()
venues_df['VenueCategory'].unique()


In [ ]:
map_kl = folium.Map(location=[lat, lng], zoom_start=11)
# Adding markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'],  kl_df['Longitude'], kl_df['Neighborhood']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=10,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_kl)
map_kl

In [ ]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
kl_onehot['Neighborhoods'] = venues_df['Neighborhood']
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]
kl_grouped=kl_onehot.groupby(["Neighborhoods"]).sum().reset_index()
print(kl_grouped.shape)
kl_grouped

(16, 155)


,Neighborhoods,,ATMBank,African Restaurant,American Restaurant,Amusement Park,Arcade,Asian Restaurant,Asian RestaurantKerala RestaurantVegan and Vegetarian Restaurant,Automotive Repair ShopCar Wash and Detail,...,Spa,Stadium,Swimming Pool,Tea Room,Thai Restaurant,Theater,Vegan and Vegetarian Restaurant,Video Store,Vintage and Thrift Store,Women's Store
0,Adyar,1,0,0,0,0,1,1,1,0,...,0,0,0,1,0,0,0,0,0,1
1,Agaram,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Anna Nagar,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arumbakkam,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Ashok Nagar,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,4,0,0,0
5,Guindy,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
6,Kodambakkam,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,1
7,Myapore,0,1,0,0,0,1,2,0,0,...,1,0,0,0,0,0,2,0,0,0
8,Royapet,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
9,Taramani,0,0,0,0,0,0,2,0,1,...,0,0,0,0,0,1,1,0,0,0


In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from matplotlib import cm,colors
len((kl_grouped[kl_grouped["Vegan and Vegetarian Restaurant"] > 0]))
kl_rest = kl_grouped[["Neighborhoods","Vegan and Vegetarian Restaurant"]]
kclusters=3
kl_clustering = kl_rest.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(kl_clustering)
kl_merged = kl_rest.copy()
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head(10)
kl_merged['Latitude'] = kl_df['Latitude']
kl_merged['Longitude'] = kl_df['Longitude']
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged
map_clusters = folium.Map(location=[13.0836939, 80.270186], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
  label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
  folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys
